In [1]:
import re
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_lg")

In [2]:
fileData=[]
with open("Output.txt", "r") as file:
    for line in file:
        x = re.sub("=", " = ", line)
        x = re.sub("  "," ", x)   
        fileData.append(x)

In [3]:
import shelve
import os

dest_file="NLP_Shelfv8//" + "Dict_Lists" + ".shlf"

shelf = shelve.open(dest_file)

dataType_number = shelf["dataType_number"]
dataType_char = shelf["dataType_char"]
dataType_list = shelf["dataType_list"]
dataType_bool = shelf["dataType_bool"]
dataType_class = shelf["dataType_class"]
dataType_void = shelf["dataType_void"]
dataType = shelf["dataType"]

variables_list = shelf["variables_list"]

scope_of_variable = shelf["scope_of_variable"]

operators = shelf["operators"]

command_declare = shelf["command_declare"]
command_print = shelf["command_print"]
command_input = shelf["command_input"]

func_prog = shelf["func_prog"]

special_class = shelf["special_class"]
special_function = shelf["special_function"]

shelf.close()

In [4]:
#Defining Patterns
#{"IN":["","s"]}
matcher = Matcher(nlp.vocab)

pattern1_1 = [{"LOWER":{"IN":dataType}}]
matcher.add("DATATYPE", None, pattern1_1)

pattern2_1 = [{"LOWER":"global"}]
pattern2_2 = [{"LOWER":"local"}]
matcher.add("SCOPE", None, pattern2_1, pattern2_2)

pattern3_1 = [{"LOWER":{"IN":operators}}]
pattern3_2 = [{"LOWER":{"IN":["additions", "adds"]}}]
pattern3_3 = [{"LOWER":{"IN":["subtractions", "subs"]}}]
pattern3_4 = [{"LOWER":{"IN":["minuses", "multiplies", "muls"]}}]
pattern3_5 = [{"LOWER":{"IN":["divides", "divs"]}}]
pattern3_6 = [{"LOWER":{"IN":["modulus", "moduluses", "mods"]}}]
matcher.add("OPERATOR", None, pattern3_1, pattern3_2, pattern3_3, pattern3_4, pattern3_5, pattern3_6)

pattern4_1 = [{"LOWER":{"IN":command_declare}}]
pattern4_2 = [{"LOWER":{"IN":command_print}}]
pattern4_3 = [{"LOWER":{"IN":command_input}}]
pattern4_4 = [{"LOWER":{"IN":special_function}}]
matcher.add("COMMAND", None, pattern4_1, pattern4_2, pattern4_3, pattern4_4)

pattern5_1 = [{"LOWER":{"IN":["function","functions"]}}]
pattern5_2 = [{"LOWER":{"IN":["func","funcs"]}}]
pattern5_3 = [{"LOWER":{"IN":["program","programs"]}}]
pattern5_4 = [{"LOWER":{"IN":["programme","programmes"]}}]
pattern5_5 = [{"LOWER":{"IN":["prog","progs"]}}]

pattern5_1 = [{"LOWER":{"IN":func_prog}}]
pattern5_2 = [{"LOWER":{"IN":["funcs","functions"]}}]
pattern5_3 = [{"LOWER":{"IN":["programs","programmes","progs"]}}]
matcher.add("FUNC_PROG", None, pattern5_1, pattern5_2, pattern5_3)

pattern6_1 = [{"LOWER": {"IN": variables_list}}]
matcher.add("VARI", None, pattern6_1)

pattern7_1 = [{"LOWER": {"IN": special_class}}]
matcher.add("SPL_CLASS", None, pattern7_1)

pattern8_1 = [{"LIKE_NUM": True}]
matcher.add("VALUE", None, pattern8_1)

In [5]:
#Creating Training Data

TRAINING_DATA = []

# Create a Doc object for each text in TEXTS
for doc in nlp.pipe(fileData):
    
    #x = re.sub("=", " = ", doc)
    #doc = re.sub("  "," ", x)
    
    # Match on the doc and create a list of matched spans
    spans = [doc[start:end] for match_id, start, end in matcher(doc)]
    
    #Modified Start
    entity_label=[]
    entity_label = [(nlp.vocab.strings[match_id]) for match_id, start, end in matcher(doc)]
    #if entity_label!=[]:
    #    print (entity_label)
    entities = [(span.start_char, span.end_char, entity_label[index]) for index,span in enumerate(spans)]
    
    #for eachWord in entities:
    #    print("Entities: {}".format(eachWord))
    
    # Format the matches as a (doc.text, entities) tuple
    training_example = (doc.text, {"tags": entities})
    
    # Append the example to the training data
    TRAINING_DATA.append(training_example)
    #Modified End
    
    '''
    entities = [(span.start_char, span.end_char, "INITIALIZATION") for span in spans]
    
    for eachWord in entities:
        print("Entities: {}".format(eachWord))
        
    # Format the matches as a (doc.text, entities) tuple
    training_example = (doc.text, {"entities": entities})
    # Append the example to the training data
    TRAINING_DATA.append(training_example)
    '''
    

#print(*TRAINING_DATA, sep="\n")


In [6]:
#Store to Data file for inspection
with open("training_data_pos.txt","w") as o_file:
    for item in TRAINING_DATA:
        o_file.write("{}\n".format(item))

In [30]:
#Training Loop
import random

#nlp = spacy.blank("en")
#pos_tagger = nlp.create_pipe("tagger")
#nlp.add_pipe(pos_tagger)

tagger = nlp.create_pipe("tagger")

tagger.add_label("DATATYPE")
tagger.add_label("SCOPE")
tagger.add_label("OPERATOR")
tagger.add_label("COMMAND")
tagger.add_label("FUNC_PROG")
tagger.add_label("VARI")
tagger.add_label("SPL_CLASS")
tagger.add_label("VALUE")

# Start the training
nlp.begin_training()

# Loop for 10 iterations
for itn in range(3):
    # Shuffle the training data
    random.shuffle(TRAINING_DATA)
    losses = {}

    # Batch the examples and iterate over them
    for batch in spacy.util.minibatch(TRAINING_DATA, size=1000):
        texts = [text for text, entities in batch]
        annotations = [entities for text, entities in batch]

        # Update the model
        nlp.update(texts, annotations, losses=losses)
    print(losses)

IndexError: list index out of range

In [35]:
TAG_MAP = {"noun": {"pos": "NOUN"}, "verb": {"pos": "VERB"}, "adj": {"pos": "ADJ"}, "adv": {"pos": "ADV"}}

#Training Loop
import random

#nlp = spacy.blank("en")
#pos_tagger = nlp.create_pipe("tagger")
#nlp.add_pipe(pos_tagger)

tagger = nlp.create_pipe("tagger")
for tag, values in TAG_MAP.items():
    tagger.add_label(tag, values)
#nlp.add_pipe(tagger)

tagger.add_label("DATATYPE")
tagger.add_label("SCOPE")
tagger.add_label("OPERATOR")
tagger.add_label("COMMAND")
tagger.add_label("FUNC_PROG")
tagger.add_label("VARI")
tagger.add_label("SPL_CLASS")
tagger.add_label("VALUE")

# Start the training
nlp.begin_training()

'''
# Loop for 10 iterations
for itn in range(3):
    # Shuffle the training data
    random.shuffle(TRAINING_DATA)
    losses = {}

    # Batch the examples and iterate over them
    for batch in spacy.util.minibatch(TRAINING_DATA, size=1000):
        texts = [text for text, entities in batch]
        annotations = [entities for text, entities in batch]

        # Update the model
        nlp.update(texts, annotations, losses=losses)
    print(losses)
'''
    
for i in range(3):
    random.shuffle(TRAINING_DATA)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(TRAINING_DATA, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        nlp.update(texts, annotations, sgd=optimizer, losses=losses)
    print("Losses", losses)

IndexError: list index out of range

In [31]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding

TAG_MAP = {"noun": {"pos": "NOUN"}, "verb": {"pos": "VERB"}, "adj": {"pos": "ADJ"}, "adv": {"pos": "ADV"}}

TRAIN_DATA = [
    ('Afrotropical', {'tags': ['adj']}), ('Afrocentricity', {'tags': ['noun']}),
    ('Afrocentric', {'tags': ['adj']}), ('Afrocentrism', {'tags': ['noun']}),
    ('Anglomania', {'tags': ['noun']}), ('Anglocentric', {'tags': ['adj']}),
    ('apraxic', {'tags': ['adj']}), ('aglycosuric', {'tags': ['adj']}),
    ('asecretory', {'tags': ['adj']}), ('aleukaemic', {'tags': ['adj']}),
    ('agrin', {'tags': ['adj']}), ('Eurotransplant', {'tags': ['noun']}),
    ('Euromarket', {'tags': ['noun']}), ('Eurocentrism', {'tags': ['noun']}),
    ('adendritic', {'tags': ['adj']}), ('asynaptic', {'tags': ['adj']}),
    ('Asynapsis', {'tags': ['noun']}), ('ametabolic', {'tags': ['adj']})
]
nlp = spacy.load("en_core_web_lg", disable=['ner', 'parser'])

tagger = nlp.create_pipe("tagger")


#nlp.vocab.vectors.name = 'spacy_pretrained_vectors'
optimizer = nlp.begin_training()
for i in range(3):
    random.shuffle(TRAIN_DATA)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        nlp.update(texts, annotations, sgd=optimizer, losses=losses)
    print("Losses", losses)

# test the trained model
test_text = "I like Afrotropical apraxic blue eggs and Afrocentricity. A Eurotransplant is cool too. The agnathostomatous Euromarket and asypnapsis is even cooler. What about Eurocentrism?"
doc = nlp(test_text)
print("Tags", [(t.text, t.tag_, t.pos_) for t in doc])


Losses {'tagger': 0.0}
Losses {'tagger': 0.0}
Losses {'tagger': 0.0}
Tags [('I', 'PRP', 'PRON'), ('like', 'VBP', 'VERB'), ('Afrotropical', 'NNP', 'PROPN'), ('apraxic', 'NN', 'NOUN'), ('blue', 'JJ', 'ADJ'), ('eggs', 'NNS', 'NOUN'), ('and', 'CC', 'CCONJ'), ('Afrocentricity', 'NNP', 'PROPN'), ('.', '.', 'PUNCT'), ('A', 'DT', 'DET'), ('Eurotransplant', 'NNP', 'PROPN'), ('is', 'VBZ', 'AUX'), ('cool', 'JJ', 'ADJ'), ('too', 'RB', 'ADV'), ('.', '.', 'PUNCT'), ('The', 'DT', 'DET'), ('agnathostomatous', 'JJ', 'ADJ'), ('Euromarket', 'NNP', 'PROPN'), ('and', 'CC', 'CCONJ'), ('asypnapsis', 'NNP', 'PROPN'), ('is', 'VBZ', 'AUX'), ('even', 'RB', 'ADV'), ('cooler', 'JJR', 'ADJ'), ('.', '.', 'PUNCT'), ('What', 'WP', 'PRON'), ('about', 'IN', 'ADP'), ('Eurocentrism', 'NN', 'NOUN'), ('?', '.', 'PUNCT')]
